In [1]:
from google.colab import files
uploaded = files.upload()

Saving News_dataset.csv to News_dataset.csv


In [2]:
import pandas as pd

df = pd.read_csv("News_dataset.csv", sep=';')
df.head()

,File_Name,Content,Category,Complete_Filename
0,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,business,001.txt-business
1,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,business,002.txt-business
2,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,business,003.txt-business
3,004.txt,High fuel prices hit BA's profits\n\nBritish A...,business,004.txt-business
4,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,business,005.txt-business


# Исследование данных

- размерность данных
- пропущенные значения
- типы данных в датасете
- количество и баланс классов

In [ ]:
df.shape

(2225, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   File_Name          2225 non-null   object
 1   Content            2225 non-null   object
 2   Category           2225 non-null   object
 3   Complete_Filename  2225 non-null   object
dtypes: object(4)
memory usage: 69.7+ KB


In [ ]:
df.isna().sum()

File_Name            0
Content              0
Category             0
Complete_Filename    0
dtype: int64

In [ ]:
df['Category'].unique()

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [3]:
df['Category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: Category, dtype: int64

# Обработка данных

Предварительная обработка и очистка текстовых данных
Любой рабочий процесс анализа данных начинается с их загрузки. Далее мы должны пропустить их через конвейер (pipeline) предобработки:

- удаление знаков препинания, лишних и спец символов
- приведение к одному регистру
- токенизировать текст – разбить текст на предложения, слова и другие единицы;
- удалить стоп-слова;
- привести слова к нормальной форме;
- векторизовать тексты – сделать числовые представления текстов для их дальнейшей обработки классификатором.

Все эти шаги служат для уменьшения шума, присущего любому обычному тексту, и повышения точности результатов классификатора. Для решения указанных задач есть несколько отличных библиотек, например, NLTK, TextBlob и spaCy. Последнюю мы и будем применять в этом руководстве.

In [4]:
df.loc[1]['Content']

'Dollar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\n\nAnd Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of conditions u

## Работа со специальными символами и знаками пунктуации

Посмотрев на один из текстов мы можем увидеть много специальных символов:

- \r
- \n - переход на новую строку
- \ before possessive pronouns (government's = government\'s)
- \ before possessive pronouns 2 (Yukos' = Yukos\')
- " when quoting text

In [5]:
df.head(2)

,File_Name,Content,Category,Complete_Filename
0,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,business,001.txt-business
1,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,business,002.txt-business


In [6]:
# \r and \n

df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df

,File_Name,Content,Category,Complete_Filename,Content_Parsed_1
0,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,business,001.txt-business,Ad sales boost Time Warner profit Quarterly p...
1,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,business,002.txt-business,Dollar gains on Greenspan speech The dollar h...
2,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,business,003.txt-business,Yukos unit buyer faces loan claim The owners ...
3,004.txt,High fuel prices hit BA's profits\n\nBritish A...,business,004.txt-business,High fuel prices hit BA's profits British Air...
4,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,business,005.txt-business,Pernod takeover talk lifts Domecq Shares in U...
...,...,...,...,...,...
2220,397.txt,BT program to beat dialler scams\n\nBT is intr...,tech,397.txt-tech,BT program to beat dialler scams BT is introd...
2221,398.txt,Spam e-mails tempt net shoppers\n\nComputer us...,tech,398.txt-tech,Spam e-mails tempt net shoppers Computer user...
2222,399.txt,Be careful how you code\n\nA new European dire...,tech,399.txt-tech,Be careful how you code A new European direct...
2223,400.txt,US cyber security chief resigns\n\nThe man mak...,tech,400.txt-tech,US cyber security chief resigns The man makin...


In [7]:
df.loc[1]['Content']

'Dollar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\n\nAnd Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of conditions u

In [8]:
df.loc[1]['Content_Parsed_1']

'Dollar gains on Greenspan speech  The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.  And Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of conditions under

In [9]:
# " уберем ковычки в тексте
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [10]:
# приведем весь текст к одному регистру
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower() #upper()

df.loc[1]['Content_Parsed_2']

"dollar gains on greenspan speech  the dollar has hit its highest level against the euro in almost three months after the federal reserve head said the us trade deficit is set to stabilise.  and alan greenspan highlighted the us government's willingness to curb spending and rising household savings as factors which may help to reduce it. in late trading in new york, the dollar reached $1.2871 against the euro, from $1.2974 on thursday. market concerns about the deficit has hit the greenback in recent months. on friday, federal reserve chairman mr greenspan's speech in london ahead of the meeting of g7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected us jobs data. i think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time, said robert sinche, head of currency strategy at bank of america in new york. he's taking a longer-term view, laying out a set of conditions under which t

In [11]:
list("?:!.,;")

['?', ':', '!', '.', ',', ';']

In [12]:
# в тексте присутвуют много знаков пунктуации, которые не свидетельсвуют о теме текста
# но если бы мы проводили анализ тональности то это могло быть важным - почему?
# что делать с пунктуацией?

punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

In [13]:
df.loc[1]['Content_Parsed_2']

"dollar gains on greenspan speech  the dollar has hit its highest level against the euro in almost three months after the federal reserve head said the us trade deficit is set to stabilise.  and alan greenspan highlighted the us government's willingness to curb spending and rising household savings as factors which may help to reduce it. in late trading in new york, the dollar reached $1.2871 against the euro, from $1.2974 on thursday. market concerns about the deficit has hit the greenback in recent months. on friday, federal reserve chairman mr greenspan's speech in london ahead of the meeting of g7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected us jobs data. i think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time, said robert sinche, head of currency strategy at bank of america in new york. he's taking a longer-term view, laying out a set of conditions under which t

In [14]:
df.loc[1]['Content_Parsed_3']

"dollar gains on greenspan speech  the dollar has hit its highest level against the euro in almost three months after the federal reserve head said the us trade deficit is set to stabilise  and alan greenspan highlighted the us government's willingness to curb spending and rising household savings as factors which may help to reduce it in late trading in new york the dollar reached $12871 against the euro from $12974 on thursday market concerns about the deficit has hit the greenback in recent months on friday federal reserve chairman mr greenspan's speech in london ahead of the meeting of g7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected us jobs data i think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time said robert sinche head of currency strategy at bank of america in new york he's taking a longer-term view laying out a set of conditions under which the current acc

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

# Удаление знаков пунктуации из текста
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ',
             41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ',
             59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ',
             93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

df['Content_Parsed_3'] = df['Content_Parsed_3'].map(lambda x: remove_punct(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

## Токенезация

**Токенизация** – это процесс разбиения текста на более мелкие части. В библиотеку spaCy уже встроен конвейер (pipeline), который начинает свою работу по обработке текста с токенизации. В этом руководстве мы разделим текст на отдельные слова. Загрузим пример и проведем разбиение:

In [ ]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
------------------------------------------------------------
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import nltk

sentence_data = "The First sentence is about Python. The Second: about Django. You can learn Python,Django and Data Ananlysis here. "
nltk_tokens = nltk.sent_tokenize(sentence_data)
print(nltk_tokens)

['The First sentence is about Python.', 'The Second: about Django.', 'You can learn Python,Django and Data Ananlysis here.']


In [ ]:
import nltk

word_data = "It originated from the idea that there are readers who prefer learning new skills from the comforts of their drawing rooms"
nltk_tokens = nltk.word_tokenize(word_data)
print(nltk_tokens)

['It', 'originated', 'from', 'the', 'idea', 'that', 'there', 'are', 'readers', 'who', 'prefer', 'learning', 'new', 'skills', 'from', 'the', 'comforts', 'of', 'their', 'drawing', 'rooms']


## Стеммнг и лемматизация

Обычно тексты содержат разные грамматические формы одного и того же слова, а также могут встречаться однокоренные слова. Лемматизация и стемминг преследуют цель привести все встречающиеся словоформы к одной, нормальной словарной форме.

**Стемминг** – это грубый эвристический процесс, который отрезает «лишнее» от корня слов, часто это приводит к потере словообразовательных суффиксов.

**Лемматизация** – это более тонкий процесс, который использует словарь и морфологический анализ, чтобы в итоге привести слово к его канонической форме – лемме.

Отличие в том, что стеммер (конкретная реализация алгоритма стемминга – прим.переводчика) действует без знания контекста и, соответственно, не понимает разницу между словами, которые имеют разный смысл в зависимости от части речи. Однако у стеммеров есть и свои преимущества: их проще внедрить и они работают быстрее. Плюс, более низкая «аккуратность» может не иметь значения в некоторых случаях.

In [ ]:
# стемминг
from nltk.stem import PorterStemmer

e_words= ["wait", "waiting", "waited", "waits"]
ps = PorterStemmer()
for w in e_words:
    rootWord=ps.stem(w)
    print(rootWord)

wait
wait
wait
wait


**Лемматизация** — это алгоритмический процесс нахождения леммы слова в зависимости от его значения. Лемматизация обычно относится к морфологическому анализу слов, целью которого является удаление флективных окончаний. Это помогает в возвращении базовой или словарной формы слова, которое известно как лемма. Метод лемматизации NLTK основан на встроенной морф-функции WorldNet. Предварительная обработка текста включает в себя как основы, так и лемматизации. Многие люди находят два термина запутанными. Некоторые относятся к ним как к одним и тем же, но между ними есть разница. Лемматизация предпочтительнее первой по следующей причине.

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()
text = "studies studying cries cry"
tokenization = nltk.word_tokenize(text)

for w in tokenization:
    print("Lemma for {} is {}".format(w, wordnet_lemmatizer.lemmatize(w)))  

Lemma for studies is study
Lemma for studying is studying
Lemma for cries is cry
Lemma for cry is cry


In [ ]:
# !pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
p = morph.parse("гуляли")
p

[Parse(word='гуляли', tag=OpencorporaTag('VERB,impf,intr plur,past,indc'), normal_form='гулять', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гуляли', 15, 10),))]

In [ ]:
p[0].normal_form

'гулять'

In [22]:
!pip install nltk
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [19]:
df.loc[0,'Content_Parsed_4']

"ad sales boost time warner profit  quarterly profits at us media giant timewarner jumped 76% to $113bn (â£600m) for the three months to december from $639m year-earlier  the firm which is now one of the biggest investors in google benefited from sales of high-speed internet connections and higher advert sales timewarner said fourth quarter sales rose 2% to $111bn from $109bn its profits were buoyed by one-off gains which offset a profit dip at warner bros and less users for aol  time warner said on friday that it now owns 8% of search-engine google but its own internet business aol had has mixed fortunes it lost 464000 subscribers in the fourth quarter profits were lower than in the preceding three quarters however the company said aol underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aol existing customers

In [23]:
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4'] # привет гуляли всю ночь - весь текст
    text_words = text.split(" ") # ['привет', 'гуляли', 'всю', 'ночь'] - список слов всего текста

    # Iterate through every word to lemmatize
    for word in text_words:
        word_norm = wordnet_lemmatizer.lemmatize(word, pos="v")
        lemmatized_list.append(word_norm) # ['привет', 'гулять', 'все', 'ночь']
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list) # привет гулять все ночь
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

df['Content_Parsed_5'] = lemmatized_text_list

In [24]:
df.loc[0,'Content_Parsed_4']
# jumped

"ad sales boost time warner profit  quarterly profits at us media giant timewarner jumped 76% to $113bn (â£600m) for the three months to december from $639m year-earlier  the firm which is now one of the biggest investors in google benefited from sales of high-speed internet connections and higher advert sales timewarner said fourth quarter sales rose 2% to $111bn from $109bn its profits were buoyed by one-off gains which offset a profit dip at warner bros and less users for aol  time warner said on friday that it now owns 8% of search-engine google but its own internet business aol had has mixed fortunes it lost 464000 subscribers in the fourth quarter profits were lower than in the preceding three quarters however the company said aol underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aol existing customers

In [25]:
df.loc[0,'Content_Parsed_5']
# jump

"ad sales boost time warner profit  quarterly profit at us media giant timewarner jump 76% to $113bn (â£600m) for the three months to december from $639m year-earlier  the firm which be now one of the biggest investors in google benefit from sales of high-speed internet connections and higher advert sales timewarner say fourth quarter sales rise 2% to $111bn from $109bn its profit be buoy by one-off gain which offset a profit dip at warner bros and less users for aol  time warner say on friday that it now own 8% of search-engine google but its own internet business aol have have mix fortunes it lose 464000 subscribers in the fourth quarter profit be lower than in the precede three quarter however the company say aol underlie profit before exceptional items rise 8% on the back of stronger internet advertise revenues it hop to increase subscribers by offer the online service free to timewarner internet customers and will try to sign up aol exist customers for high-speed broadband timewar

# Стоп-слова

Стоп-слова – это слова, которые выкидываются из текста до/после обработки текста. Когда мы применяем машинное обучение к текстам, такие слова могут добавить много шума, поэтому необходимо избавляться от нерелевантных слов.

Стоп-слова это обычно понимают артикли, междометия, союзы и т.д., которые не несут смысловой нагрузки. При этом надо понимать, что не существует универсального списка стоп-слов, все зависит от конкретного случая.

В NLTK есть предустановленный список стоп-слов. Перед первым использованием вам понадобится его скачать: nltk.download(“stopwords”). После скачивания можно импортировать пакет stopwords и посмотреть на сами слова

In [31]:
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
# st = ['stop1', 'stop2', ...]

# Loading the stop words in english
stop_words = list(stopwords.words('english'))
stop_words[0:100]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once']

In [32]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [33]:
df.loc[0,'Content_Parsed_5']

"ad sales boost time warner profit  quarterly profit at us media giant timewarner jump 76% to $113bn (â£600m) for the three months to december from $639m year-earlier  the firm which be now one of the biggest investors in google benefit from sales of high-speed internet connections and higher advert sales timewarner say fourth quarter sales rise 2% to $111bn from $109bn its profit be buoy by one-off gain which offset a profit dip at warner bros and less users for aol  time warner say on friday that it now own 8% of search-engine google but its own internet business aol have have mix fortunes it lose 464000 subscribers in the fourth quarter profit be lower than in the precede three quarter however the company say aol underlie profit before exceptional items rise 8% on the back of stronger internet advertise revenues it hop to increase subscribers by offer the online service free to timewarner internet customers and will try to sign up aol exist customers for high-speed broadband timewar

In [34]:
df.loc[0,'Content_Parsed_6']

"ad sales boost time warner profit  quarterly profit  us media giant timewarner jump 76%  $113bn (â£600m)   three months  december  $639m year-earlier   firm    one   biggest investors  google benefit  sales  high-speed internet connections  higher advert sales timewarner say fourth quarter sales rise 2%  $111bn  $109bn  profit  buoy  one- gain  offset  profit dip  warner bros  less users  aol  time warner say  friday     8%  search-engine google    internet business aol   mix fortunes  lose 464000 subscribers   fourth quarter profit  lower    precede three quarter however  company say aol underlie profit  exceptional items rise 8%   back  stronger internet advertise revenues  hop  increase subscribers  offer  online service free  timewarner internet customers   try  sign  aol exist customers  high-speed broadband timewarner also   restate 2000  2003 result follow  probe   us securities exchange commission (sec)   close  conclude  time warner fourth quarter profit  slightly better  ana

In [35]:
df.head()

,File_Name,Content,Category,Complete_Filename,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,business,001.txt-business,Ad sales boost Time Warner profit Quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...
1,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,business,002.txt-business,Dollar gains on Greenspan speech The dollar h...,dollar gains on greenspan speech the dollar h...,dollar gains on greenspan speech the dollar h...,dollar gains on greenspan speech the dollar h...,dollar gain on greenspan speech the dollar ha...,dollar gain greenspan speech dollar hit h...
2,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,business,003.txt-business,Yukos unit buyer faces loan claim The owners ...,yukos unit buyer faces loan claim the owners ...,yukos unit buyer faces loan claim the owners ...,yukos unit buyer faces loan claim the owners ...,yukos unit buyer face loan claim the owners o...,yukos unit buyer face loan claim owners emb...
3,004.txt,High fuel prices hit BA's profits\n\nBritish A...,business,004.txt-business,High fuel prices hit BA's profits British Air...,high fuel prices hit ba's profits british air...,high fuel prices hit ba's profits british air...,high fuel prices hit ba profits british airwa...,high fuel price hit ba profit british airways...,high fuel price hit ba profit british airways...
4,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,business,005.txt-business,Pernod takeover talk lifts Domecq Shares in U...,pernod takeover talk lifts domecq shares in u...,pernod takeover talk lifts domecq shares in u...,pernod takeover talk lifts domecq shares in u...,pernod takeover talk lift domecq share in uk ...,pernod takeover talk lift domecq share uk dr...


In [38]:
list_columns = ["Category", "Content", "Content_Parsed"]
# df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
df = df[list_columns]
df.head()

,Category,Content,Content_Parsed
0,business,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit quarterly p...
1,business,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech dollar hit h...
2,business,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim owners emb...
3,business,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profit british airways...
4,business,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq share uk dr...


# Подготовка данных для обучения

In [39]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

df.head()

,Category,Content,Content_Parsed,Category_Code
0,business,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit quarterly p...,0
1,business,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech dollar hit h...,0
2,business,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim owners emb...,0
3,business,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profit british airways...,0
4,business,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq share uk dr...,0


In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

# Векторизация текста

Text representation
We have various options:

- Count Vectors as features
- TF-IDF Vectors as features
- Word Embeddings as features
- Text / NLP based features
- Topic Models as features

We'll use TF-IDF Vectors as features.

We have to define the different parameters:

- ngram_range: We want to consider both unigrams and bigrams.
- max_df: When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold
- min_df: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.
- max_features: If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

See TfidfVectorizer? for further detail.

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

ngram_range = (1,2)
# словные
# униграммы, биграммы

min_df = 10
max_df = 1.
max_features = 300

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray() # fit()-обучение transfrom()-применение
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 300)
(334, 300)


In [42]:
feature_names = tfidf.get_feature_names()
print("Feature names", feature_names)

Feature names ['10', '2003', '2004', '2005', 'able', 'accord', 'act', 'action', 'add', 'allow', 'already', 'also', 'although', 'announce', 'another', 'around', 'ask', 'attack', 'award', 'away', 'back', 'bank', 'bbc', 'beat', 'become', 'begin', 'believe', 'best', 'better', 'big', 'biggest', 'bill', 'blair', 'book', 'break', 'bring', 'britain', 'british', 'brown', 'build', 'business', 'buy', 'call', 'campaign', 'case', 'chance', 'change', 'charge', 'chief', 'china', 'claim', 'close', 'club', 'come', 'company', 'computer', 'concern', 'continue', 'control', 'cost', 'could', 'countries', 'country', 'court', 'create', 'cut', 'data', 'day', 'deal', 'decision', 'demand', 'despite', 'digital', 'director', 'drive', 'earlier', 'economic', 'economy', 'election', 'end', 'england', 'eu', 'europe', 'european', 'even', 'every', 'executive', 'expect', 'face', 'fail', 'fall', 'far', 'figure', 'file', 'film', 'final', 'find', 'firm', 'first', 'five', 'follow', 'force', 'foreign', 'former', 'four', 'franc

In [44]:
df2 = pd.DataFrame(features_train, columns=feature_names)
df2.head()

,10,2003,2004,2005,able,accord,act,action,add,allow,already,also,although,announce,another,around,ask,attack,award,away,back,bank,bbc,beat,become,begin,believe,best,better,big,biggest,bill,blair,book,break,bring,britain,british,brown,build,...,talk,tax,team,technology,tell,tell bbc,term,test,think,third,three,time,title,top,trade,try,turn,tv,two,uk,unite,us,use,users,video,vote,wales,want,warn,way,week,well,win,without,work,world,would,year,year old,years
0,0.000000,0.000000,0.000000,0.000000,0.119576,0.000000,0.000000,0.000000,0.081110,0.000000,0.000000,0.000000,0.117200,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.103211,0.199844,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.24735,0.111843,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.091924,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.062519,0.000000,0.0,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.164515,0.109074,0.000000,0.056847,0.000000,0.0,0.0,0.0,0.0,0.329047,0.0,0.114899,0.000000,0.127717,0.000000,0.0,0.000000,0.000000,0.101666,0.0,0.0,0.0,0.0,0.000000,0.216645,0.0,0.0,0.000000,0.00000,0.226852,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.072133,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.08218,0.135908,0.060424,0.000000,0.0,0.000000
2,0.102259,0.000000,0.000000,0.000000,0.000000,0.104321,0.000000,0.000000,0.078220,0.000000,0.101328,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.079521,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,...,0.112085,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.113597,0.0,0.0,0.0,0.0,0.0,0.0,0.070322,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.138838,0.000000,0.000000,0.0,0.085025,0.0,0.0,0.00000,0.000000,0.060291,0.059050,0.0,0.000000
3,0.000000,0.193843,0.279994,0.118175,0.000000,0.000000,0.000000,0.000000,0.084602,0.000000,0.000000,0.128748,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.137579,0.000000,0.0,0.0,0.119753,0.00000,0.000000,0.0,0.0,...,0.000000,0.300114,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.131806,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.08869,0.000000,0.000000,0.178303,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.199977,0.108741,0.000000,0.103972,0.000000,0.113719,0.107975,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.149902,0.0,0.000000,0.000000,0.164085,0.0,0.0,0.0,0.0,0.121518,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.129447,0.175812,0.185173,0.0,0.0,0.099187,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.076637,0.0,0.0,0.0,0.130983,0.078337,0.110326,0.083546,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.057598,0.118387,0.0,0.160063


# Подбор моделей

После того, как мы построим наши векторы признаков, мы попробуем несколько моделей классификации машинного обучения, чтобы найти, какая из них лучше всего работает с нашими данными. Мы попробуем со следующими моделями:

- Random Forest
- Support Vector Machine (SVC)
- K Nearest Neighbors (KNN)
- Multinomial Logistic Regression
- Gradient Boosting

- как подобрать гиперпараметры модели?
- как метрики для оценки модели использовать?
- нужна ли нормализация данных?

In [45]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

svc_base = svm.SVC(random_state=8)

print('Parameters currently in use:\n')
pprint(svc_base.get_params())

Parameters currently in use:

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 8,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [46]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [47]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

- C: Penalty parameter C of the error term.
- kernel: Specifies the kernel type to be used in the algorithm.
- gamma: Kernel coefficient.
- degree: Degree of the polynomial kernel function.

## Подбор гиперпараметров модели

In [ ]:
# C
C = [.0001, .001, .01]

# gamma
gamma = [.0001, .001, .01, .1, 1, 10, 100]

# degree
degree = [1, 2, 3, 4, 5]

# kernel
kernel = ['linear', 'rbf', 'poly']

# probability
probability = [True]

# Create the random grid
random_grid = {'C': C,
              'kernel': kernel,
              'gamma': gamma,
              'degree': degree,
              'probability': probability
             }

pprint(random_grid)

{'C': [0.0001, 0.001, 0.01],
 'degree': [1, 2, 3, 4, 5],
 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
 'kernel': ['linear', 'rbf', 'poly'],
 'probability': [True]}


In [ ]:
# First create the base model to tune
svc = svm.SVC(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=svc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(features_train, labels_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 15.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=8,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=50, n_jobs=None,
                   param_distributions={'C': [0.0001, 0.001, 0.01],
                                        'degree': [1, 2, 3, 4, 5],
                                        'gamma': [0.0001, 0.001, 0.01, 0.1, 1,
                                                  10, 100],
                                        'kernel': ['linear', 'rbf', 'poly'],
                                        'probability': [True]},
                   pre_dispatch='2*n_jobs', random_s

In [ ]:
# После перебора гиперпараметров, посмотрим которые лучшие

print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

The best hyperparameters from Random Search are:
{'probability': True, 'kernel': 'poly', 'gamma': 10, 'degree': 4, 'C': 0.01}

The mean accuracy of a model with these hyperparameters is:
0.9238497723442256


In [ ]:
best_svc = random_search.best_estimator_

best_svc.fit(features_train, labels_train)

best_svc_pred = best_svc.predict(features_test)

In [ ]:
# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(labels_train, best_svc.predict(features_train)))

The training accuracy is: 
1.0


In [ ]:
# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(labels_test, best_svc_pred))

The test accuracy is: 
0.9281437125748503


In [ ]:
base_model = svm.SVC(random_state = 8)
base_model.fit(features_train, labels_train)
accuracy_score(labels_test, base_model.predict(features_test))

0.9491017964071856

In [ ]:
best_svc.fit(features_train, labels_train)
accuracy_score(labels_test, best_svc.predict(features_test))

0.9281437125748503